<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# Hyperparameter-Tuning mit PyTorch Lightning und Optuna

## a) Import der benötigten Bibliotheken
In diesem Schritt bereiten wir unser Arbeitsumfeld vor, indem wir die benötigten Bibliotheken importieren und den **FashionMNIST-Datensatz** laden. 
FashionMNIST ist ein Standard-Bildklassifizierungsdatensatz mit Graustufenbildern von Modeartikeln (Schuhe, Hosen, T-Shirts etc.), die in **10 verschiedene Klassen** unterteilt sind. 
Die **LightningDataModule**-Struktur hilft uns später, die Daten einfacher zu verwalten.

In [ ]:
import os
from typing import List, Optional

#!pip install optuna
#!pip install optuna-integration[pytorch_lightning]

import lightning.pytorch as pl
import optuna
import torch
import torch.nn.functional as F
from optuna.integration import PyTorchLightningPruningCallback
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


### Allgemeine Einstellungen

In [ ]:
PERCENT_VALID_EXAMPLES = (
    0.1  # 10 % der Trainingsdaten werden für die Validierung genutzt
)
BATCHSIZE = 128  # Anzahl der Samples pro Batch
CLASSES = 10  # 10 Klassen für FashionMNIST
EPOCHS = 10  # Maximale Anzahl der Trainings-Epochen
TRIALS = 10  # Anzahl der Versuche für Optuna
DIR = os.getcwd()  # Arbeitsverzeichnis


## b) Definition des neuronalen Netzwerks
Hier definieren wir unser **neuronales Netzwerk (MLP-Modell)**. Es besteht aus:
- Einer **Flatten-Schicht**, die das 28×28-Bild in einen Vektor umwandelt.
- Einer **voll verbundenen (Dense) Schicht** mit einer variablen Anzahl an Neuronen.
- Einer **ReLU-Aktivierungsfunktion**, um Nichtlinearität hinzuzufügen.
- Einer **Dropout-Schicht**, um Überanpassung (Overfitting) zu reduzieren.
- Einer **voll verbundenen Ausgabe-Schicht**, die das Bild in eine der **10 Klassen** klassifiziert.

Die Anzahl der **Neuronen** in der ersten Schicht und die **Dropout-Rate** sind Hyperparameter, die später optimiert werden.

In [ ]:
class Net(nn.Module):
    def __init__(self, dropout: float, hidden_units: int) -> None:
        super().__init__()
        self.flatten = nn.Flatten()  # Wandelt das 28x28 Bild in einen Vektor um
        self.fc1 = nn.Linear(28 * 28, hidden_units)  # Erste vollverbundene Schicht
        self.relu = nn.ReLU()  # Aktivierungsfunktion
        self.dropout = nn.Dropout(dropout)  # Dropout-Schicht für Regularisierung
        self.fc2 = nn.Linear(hidden_units, CLASSES)  # Ausgabe-Schicht mit 10 Klassen

    def forward(self, data: torch.Tensor) -> torch.Tensor:
        x = self.flatten(data)  # Bild in Vektor umwandeln
        x = self.fc1(x)  # Durch die erste vollverbundene Schicht
        x = self.relu(x)  # Aktivierungsfunktion anwenden
        x = self.dropout(x)  # Dropout für Regularisierung
        x = self.fc2(x)  # Durch die Ausgabe-Schicht
        return F.log_softmax(x, dim=1)  # Log-Softmax für Klassifikation anwenden


## c) Definition des Lightning-Moduls
Ein **LightningModule** kapselt das Training, die Validierung und das Testen eines Modells. 
- `training_step()` berechnet den Verlust während des Trainings.
- `validation_step()` berechnet die Genauigkeit auf den Validierungsdaten.
- `configure_optimizers()` gibt den Optimierer (Adam) zurück.

In [ ]:
class LightningNet(pl.LightningModule):
    def __init__(self, dropout: float, hidden_units: int, learning_rate: float) -> None:
        super().__init__()
        self.model = Net(
            dropout, hidden_units
        )  # Initialisierung des neuronalen Netzwerks
        self.learning_rate = learning_rate  # Lernrate speichern

    def forward(self, data: torch.Tensor) -> torch.Tensor:
        return self.model(data)  # Vorwärtsdurchlauf des Modells

    def training_step(self, batch: List[torch.Tensor], batch_idx: int) -> torch.Tensor:
        data, target = batch  # Eingabedaten und Zielwerte aus dem Batch extrahieren
        output = self(data)  # Modellvorhersage
        loss = F.nll_loss(output, target)  # Berechnung des Verlusts
        self.log("train_loss", loss, prog_bar=True)  # Verlust im Trainingslog speichern
        return loss  # Verlust zurückgeben

    def validation_step(self, batch: List[torch.Tensor], batch_idx: int) -> None:
        data, target = batch  # Eingabedaten und Zielwerte aus dem Batch extrahieren
        output = self(data)  # Modellvorhersage
        pred = output.argmax(dim=1, keepdim=True)  # Vorhersagen in Klassen umwandeln
        accuracy = pred.eq(target.view_as(pred)).float().mean()  # Genauigkeit berechnen
        self.log(
            "val_acc", accuracy, prog_bar=True
        )  # Genauigkeit im Validierungslog speichern
        self.log(
            "hp_metric", accuracy, on_step=False, on_epoch=True, prog_bar=True
        )  # Genauigkeit als Hyperparameter-Metrik speichern

    def configure_optimizers(self) -> optim.Optimizer:
        return optim.Adam(
            self.model.parameters(), lr=self.learning_rate
        )  # Optimierer konfigurieren

    def test_step(self, batch: List[torch.Tensor], batch_idx: int) -> None:
        data, target = batch  # Eingabedaten und Zielwerte aus dem Batch extrahieren
        output = self(data)  # Modellvorhersage
        pred = output.argmax(dim=1, keepdim=True)  # Vorhersagen in Klassen umwandeln
        accuracy = pred.eq(target.view_as(pred)).float().mean()  # Genauigkeit berechnen
        self.log(
            "test_acc", accuracy, prog_bar=True
        )  # Genauigkeit im Testlog speichern


## d) DataLoader für FashionMNIST
Hier implementieren wir eine Klasse, um den FashionMNIST-Datensatz effizient zu verwalten. 
- **Trainings-, Validierungs- und Test-Sets** werden erstellt.
- **Dataloader** versorgen das Modell mit den richtigen Batch-Grössen.

In [ ]:
class FashionMNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str, batch_size: int):
        super().__init__()
        self.data_dir = data_dir  # Verzeichnis für die Daten
        self.batch_size = batch_size  # Batch-Grösse

    def setup(self, stage: Optional[str] = None) -> None:
        # Laden des Test-Datensatzes
        self.mnist_test = datasets.FashionMNIST(
            self.data_dir, train=False, download=True, transform=transforms.ToTensor()
        )
        # Laden des vollständigen Trainings-Datensatzes
        mnist_full = datasets.FashionMNIST(
            self.data_dir, train=True, download=True, transform=transforms.ToTensor()
        )
        # Aufteilen des Trainings-Datensatzes in Trainings- und Validierungs-Datensatz
        self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

    def train_dataloader(self) -> DataLoader:
        # DataLoader für den Trainings-Datensatz
        return DataLoader(
            self.mnist_train, batch_size=self.batch_size, shuffle=True, pin_memory=True
        )

    def val_dataloader(self) -> DataLoader:
        # DataLoader für den Validierungs-Datensatz
        return DataLoader(
            self.mnist_val, batch_size=self.batch_size, shuffle=False, pin_memory=True
        )

    def test_dataloader(self) -> DataLoader:
        # DataLoader für den Test-Datensatz
        return DataLoader(
            self.mnist_test, batch_size=self.batch_size, shuffle=False, pin_memory=True
        )


## e) Definition der Optuna Optimierung
Hier definieren wir eine `objective`-Funktion für Optuna, die verschiedene Kombinationen von Hyperparametern testet, um das Modell zu optimieren. 
Optuna variiert **Dropout, Anzahl der Neuronen und die Lernrate**, um die beste Leistung zu finden.

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    # Vorschlagen der Anzahl der Neuronen in der versteckten Schicht
    hidden_units = trial.suggest_int("units", 32, 128, step=32)
    # Vorschlagen der Dropout-Rate
    dropout = trial.suggest_float("dropout", 0.2, 0.5)
    # Vorschlagen der Lernrate
    learning_rate = trial.suggest_categorical("learning_rate", [1e-2, 1e-3, 1e-4])

    # Initialisieren des Modells mit den vorgeschlagenen Hyperparametern
    model = LightningNet(dropout, hidden_units, learning_rate)
    # Initialisieren des DataModules
    datamodule = FashionMNISTDataModule(data_dir=DIR, batch_size=BATCHSIZE)

    # Initialisieren des Trainers
    trainer = pl.Trainer(
        logger=True,  # Logger aktivieren
        limit_val_batches=PERCENT_VALID_EXAMPLES,  # Begrenzen der Validierungs-Batches
        enable_checkpointing=False,  # Checkpointing deaktivieren
        max_epochs=EPOCHS,  # Maximale Anzahl der Epochen
        accelerator="auto",  # Automatische Auswahl der Hardware (CPU/GPU)
        devices=[0],  # Verwenden des ersten Geräts (z.B. GPU 0)
        callbacks=[
            # Callback für das Pruning (frühes Beenden) basierend auf der Validierungsgenauigkeit
            PyTorchLightningPruningCallback(trial, monitor="val_acc"),
            # Callback für das frühe Stoppen basierend auf der Validierungsgenauigkeit
            pl.callbacks.EarlyStopping(monitor="val_acc", patience=5),
        ],
    )
    # Training des Modells
    trainer.fit(model, datamodule=datamodule)

    # Rückgabe der besten Validierungsgenauigkeit
    return trainer.callback_metrics.get("val_acc", torch.tensor(0)).item()


## f) Hyperparametersuche ausführen
Jetzt starten wir die Hyperparameter-Optimierung mit **Optuna**. Optuna testet mehrere Kombinationen von Hyperparametern und bestimmt automatisch die beste Konfiguration für unser Modell.
- `study = optuna.create_study(direction="maximize")`: Erstellt eine Optuna-Studie zur Optimierung der **Validierungsgenauigkeit**.
- `study.optimize(objective, n_trials=TRIALS)`: Führt `TRIALS` viele Experimente durch, um die beste Hyperparameter-Kombination zu finden.
- `study.best_params`: Gibt die besten gefundenen Werte für die Hyperparameter aus.

In [ ]:
# Erstellen eines Optuna-Studiums zur Hyperparameter-Optimierung
study = optuna.create_study(direction="maximize")
# Durchführen der Versuche und Ausgabe des aktuellen Versuchs
for i, trial in enumerate(study.trials, start=1):
    print(f"Running trial {i}/{TRIALS}")
# Optimieren des Modells mit der definierten Zielsetzung
study.optimize(objective, n_trials=TRIALS, timeout=600)


## g) Training mit den optimalen Hyperparametern
Nachdem Optuna die besten Hyperparameter gefunden hat, trainieren wir das Modell erneut mit diesen Werten.
- Die besten Hyperparameter werden aus `study.best_trial.params` extrahiert.
- Das Modell wird mit diesen optimalen Parametern initialisiert.
- Ein neuer `Trainer` trainiert das Modell erneut mit den besten Hyperparametern.

In [ ]:
# Abrufen der besten Hyperparameter
best_hps = study.best_trial.params
print("Best hyperparameters:")
# Ausgabe der besten Hyperparameter
for key, value in best_hps.items():
    print(f"{key}: {value}")

# Initialisieren des Modells mit den besten Hyperparametern
best_model = LightningNet(
    dropout=best_hps["dropout"],
    hidden_units=best_hps["units"],
    learning_rate=best_hps["learning_rate"],
)
# Initialisieren des DataModules
datamodule = FashionMNISTDataModule(data_dir=DIR, batch_size=BATCHSIZE)
# Initialisieren des Trainers
trainer = pl.Trainer(max_epochs=EPOCHS, accelerator="auto", devices=[0])
# Training des Modells mit den besten Hyperparametern
trainer.fit(best_model, datamodule=datamodule)


## h) Evaluieren Sie das Modell auf den Testdaten
Der letzte Schritt besteht darin, unser trainiertes Modell auf den Testdaten zu evaluieren, um zu überprüfen, wie gut es generalisiert.
- `trainer.test()` wertet das Modell auf dem Testdatensatz aus.
- `print("Test results:", test_results)` gibt die Testergebnisse aus, einschließlich der finalen Genauigkeit des Modells.

In [ ]:
# Ausgabe des besten Modells
print(best_model)
# Testen des Modells und Ausgabe der Testergebnisse
test_results = trainer.test(best_model, datamodule=datamodule)
print("Test results:", test_results)


## Fazit
Mit dieser Übung haben Sie gelernt, wie man **PyTorch Lightning** und **Optuna** kombiniert, um ein neuronales Netz für Bildklassifikation zu optimieren. Durch **automatisches Hyperparameter-Tuning** können Sie die Leistung Ihres Modells erheblich verbessern. 🎯🚀